In [307]:
#---------------------------------importing libaries ------------------------------------------#
import numpy as np
import json
import pandas as pd
import ast
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable
from torch.utils.data import DataLoader


In [511]:
#------------------------------------Loading dataset --------------------------------------#
def load_dataset():
    with open('datasets/dataset.json') as json_file:  
        data = json.load(json_file)
    #print(type(data['21717']))
    #print(type(data))
    labels=[]
    for line in data['21717']:
        labels.append(line)
    ids=[]
    for ide in data.keys():
        ids.append(ide)
    return data

In [513]:
def load_kb():
    #-------------------------Loading Knowledge base ---------------------------#
    facts=pd.read_csv("datasets/KGfacts/KGfacts-CloseWorld.csv", error_bad_lines=False)
    facts.columns=["entity","relation","attribute"]
    
    #----------------Loading mapping between Qids and entities -----------------#
    df = pd.read_csv('datasets/KGfacts/Qid-NamedEntityMapping.csv',sep='\t',error_bad_lines=False, encoding='ascii')
    df.columns=["id","name"]
    names=df["name"].apply(ast.literal_eval).apply(bytes.decode)
    df["name"]=list(names)
    mapping=df
    return facts,mapping

In [514]:
def get_facts(NamedEntity):
    fc=[]
    for entity in NamedEntity:
        qid=mapping[mapping['name']==entity]
        ide=qid["id"]
        ide=pd.DataFrame(ide)
    #print(ide)
        s_find=str(ide['id']).split("\n")[0]
        index=s_find.find("Q")
        s_find=s_find[index:]
    #print(s_find)
        rel_facts=facts[facts['entity']==s_find]
    #print(rel_facts)
        rel_facts_list=[]
        for index,fact in rel_facts.iterrows():
            rel_facts_list.append([fact["entity"],fact["relation"],fact["attribute"]])    
    #print(rel_facts_list)
        rel=[]
        for fact in rel_facts_list:
            rel.append(" ".join(fact))
    #print(rel)
        rel1=[]
        for fact in rel:
            rel1.append(fact.split(" "))
        fc+=rel1
    return(fc)
    

In [515]:
def get_emb_facts(rel_facts):
    embedded_facts=[]
    for fact in rel_facts:
        em=[]
        for word in fact:  
            if word in embeddings_index:    
                em.append( embeddings_index[word])     
        embedded_facts.append(em)
    return embedded_facts

In [516]:
def  get_final_avg(embedded_facts):
    embedded_facts_arr=[]
    for fact in embedded_facts:
        embedded_facts_arr.append(np.array(fact))
    embedded_facts_arr=np.array(embedded_facts_arr)
    embedded_facts_arr_avg=[]
    for fact in embedded_facts_arr:
        embedded_facts_arr_avg.append(np.mean(fact,axis=0))
    embedded_facts_arr_avg=np.array(embedded_facts_arr_avg)
    return embedded_facts_arr_avg


In [517]:
def load_embedding_matrix():
    embeddings_index = {}
    f = open('glove.6B.100d.txt')
    for line in f:
        values = line.split(' ')
        word = values[0] ## The first entry is the word
        coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
        embeddings_index[word] = coefs
    f.close()
    print('GloVe data loaded') 
    return embeddings_index
    
    

In [518]:
#------------------------------Loading data --------------------------------------#


data=load_dataset()
#------------------------------Loading knowledge base and mapping --------------------------------------#

facts,mapping=load_kb()
#------------------------------Loading embedding matrix --------------------------------------#

embeddings_index=load_embedding_matrix()


#------------------------------one time deal--------------------------------------#


b'Skipping line 17: expected 3 fields, saw 4\nSkipping line 54: expected 3 fields, saw 4\nSkipping line 187: expected 3 fields, saw 4\nSkipping line 245: expected 3 fields, saw 4\nSkipping line 299: expected 3 fields, saw 4\nSkipping line 333: expected 3 fields, saw 4\nSkipping line 391: expected 3 fields, saw 4\nSkipping line 416: expected 3 fields, saw 4\nSkipping line 422: expected 3 fields, saw 4\nSkipping line 432: expected 3 fields, saw 4\nSkipping line 527: expected 3 fields, saw 4\nSkipping line 554: expected 3 fields, saw 4\nSkipping line 660: expected 3 fields, saw 4\nSkipping line 661: expected 3 fields, saw 4\nSkipping line 706: expected 3 fields, saw 4\nSkipping line 820: expected 3 fields, saw 4\nSkipping line 825: expected 3 fields, saw 5\nSkipping line 833: expected 3 fields, saw 4\nSkipping line 887: expected 3 fields, saw 4\nSkipping line 932: expected 3 fields, saw 4\nSkipping line 947: expected 3 fields, saw 4\nSkipping line 996: expected 3 fields, saw 4\nSkipping l

GloVe data loaded


In [520]:
#------------------------------Doinf for one example --------------------------------------#

NamedEntity= ['Barry Fitzgerald',
  'Charles B. Fitzsimons',
  'Sean McClory',
  'John Wayne',
  "Maureen O'Hara"]

#------------------------------getting relevant facts--------------------------------------#
fv=get_facts(NamedEntity)

#------------------------------getting embedding of relevant facts--------------------------------------#
embedded_facts=get_emb_facts(fv)

#------------------------------getting mean embedding of relevant facts--------------------------------------#
emded_mean=get_final_avg(embedded_facts)

# Getting Embedding of questions

In [306]:
question="Who is the person in the picture"

In [308]:
# dats is stored in form of dictionary of dictionaries.
#outer dictionary is index: {data[index]}
#inner dictionary is named_entities:[---,---,---],questions:[---,---,],....
# we have to get index for each word
#------------------------------getting inidices of data dictionary -----------------------------------#

indices=list(data.keys())
indices.sort()
train_indices=indices[:20000]
test_indices=indices[20000:]

In [310]:
#------------------------------class for indexing all words -----------------------------------#

class vocab:
    def __init__(self):
        self.n_words=0
        self.word2index={}
        self.word2count={}
        self.index2word={}
    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word]=self.n_words
            self.word2count[word]=1
            self.index2word[self.n_words]=word
            self.n_words=self.n_words+1
        else:
            self.word2count[word]+=1
        
    def index_words(self, sentence):
        for word in sentence:
            self.index_word(word)


In [311]:
#-----------------function for getting the data in right format so we can use above class ------------------#
def index_data():
    data_class=vocab()
    pre_data=[]
    for index in train_indices:
        for word in str(data[index]).split(" "):
            pre_data.append((word))
    clean_data=[]
    for word in pre_data:
        clean_data.append(word.replace(':',' ').replace(',',' ').replace('[',' ').replace('{',' ').replace(']',' ').replace('}',' ').replace('"','').replace("\\","").replace("'",""))
    data_class.index_words(clean_data)
    return data_class
    
        
        
        

In [312]:
def index_from_sentence(sentence):
    indexes=[data_class.word2index[word] for word in sentence.split(" ")]
    return indexes

In [313]:
def variable_from_sentence(sentence):
    index=index_from_sentence(sentence)
    
    var = Variable(torch.LongTensor(index).view(-1, 1))
    return var

In [314]:
# index data
process_data=index_data()
#getting tensor from indexes of an example question
var=variable_from_sentence(question)

In [380]:
question="Who is the person in the picture"
hidden_size=100
vocab_size=data_class.n_words
word_embedding=nn.Embedding(vocab_size,hidden_size,padding_idx=0,sparse=True)


In [381]:
def question_module(var,word_embedding):
    gru=nn.GRU(hidden_size,hidden_size,bidirectional=False,batch_first=True)
    #print("before any embedding",var.size())
    #print("before any embedding, the tensor is:",var)
    
    var = word_embedding(var)
    #print("after word embedding embedding",var.size())
    #print("after embedding, the tensor is:",var)
    
    bls, questions = gru(var)
    #print("after gru",questions.size())
    question_rep=torch.mean(wer,dim=0)
#questions = questions.transpose(0, 1)
    return question_rep

In [382]:
question_rep=question_module(var,word_embedding)
final_facts=torch.as_tensor(embedded_facts_arr_avg)

In [383]:
question_rep

tensor([ 0.0141,  0.0911,  0.1821, -0.0323, -0.1940,  0.0725, -0.0190, -0.0029,
        -0.0234, -0.2315,  0.0794, -0.0777, -0.0116, -0.0012,  0.0644,  0.0358,
         0.0598, -0.0477,  0.0056,  0.0879, -0.0878, -0.0498, -0.1441,  0.1717,
        -0.0134, -0.0728, -0.0477,  0.2358,  0.0683,  0.0187, -0.0287, -0.0957,
         0.0712, -0.0571, -0.0650,  0.1184, -0.1814, -0.0741, -0.0702,  0.0065,
         0.0824,  0.0392,  0.1298, -0.0784, -0.0018,  0.2101,  0.1296, -0.2299,
        -0.0465, -0.0089, -0.0157, -0.1230,  0.0131,  0.0930,  0.0774, -0.0255,
        -0.1469, -0.1431, -0.0762, -0.0583,  0.1448,  0.0998,  0.1846,  0.1257,
         0.1272,  0.2337,  0.0249, -0.1079,  0.0400, -0.0459, -0.0907, -0.0735,
         0.0329,  0.0825, -0.2087,  0.1153,  0.0652,  0.2206,  0.1038, -0.0391,
         0.1151, -0.0668, -0.0200, -0.1341, -0.0564, -0.1161, -0.0120, -0.1061,
        -0.1499, -0.0085,  0.0689, -0.1046,  0.0569,  0.0129, -0.0674,  0.1510,
         0.0479,  0.0109, -0.0775, -0.10

In [398]:
cos = nn.CosineSimilarity()


In [523]:
final_facts[0]

tensor([ 0.3301, -0.2589,  0.1292, -0.0648, -0.2591,  0.2132,  0.6387,  0.6393,
         0.1905,  0.9139,  0.0060, -0.3799,  0.2677,  1.0973, -0.4382, -1.0627,
         1.3068, -0.1554,  0.2435,  1.0565, -0.7058,  0.3902, -0.4019,  0.6487,
         0.3928,  0.4166, -0.5415, -0.0471,  1.1007,  0.4735, -0.0987,  0.1336,
         0.8668,  0.7820, -1.5777,  0.2915,  0.0008,  0.1177, -0.5003, -0.1277,
        -0.2888,  0.1175, -0.5712, -0.1256, -0.1508,  0.0031,  0.3045,  0.7384,
         0.1053,  0.3574, -0.3122, -0.0977,  0.5057, -0.2791, -0.1661, -0.7896,
         0.7270, -0.2666,  0.0180,  0.0861, -0.4589,  0.3203,  0.2126, -0.3596,
        -0.5425,  0.0201,  0.2715,  0.4108,  0.1081,  0.1808,  0.1178,  0.4051,
         0.0992, -0.5034,  0.1427,  0.4013,  0.0959,  0.2076,  0.3070, -0.0297,
         0.1513,  0.2089, -0.4619, -0.7064, -0.2686, -0.1432, -0.0857, -0.7484,
         0.4347, -0.7894, -0.3720,  0.2452,  0.0434,  0.7950, -0.0391, -0.0271,
        -0.5943, -0.1690, -0.0408,  0.22

In [421]:
final_facts[0].norm(mean=[-0.5/0.5], std=[1/0.5])

TypeError: norm() missing 1 required positional arguments: "dim"

In [416]:
q=torch.normal(mean=0.0, std=question_rep, out=None)
f=torch.normal(mean=0.0, std=final_facts[0], out=None)

In [412]:
output = cos(q, f)

RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [420]:
f

tensor([-0.2356,  0.0266,  0.0736,  0.0031,  0.2581, -0.0339, -0.0281,  0.3478,
        -0.0875, -0.4883,  0.0028, -0.1502, -0.0289, -0.1239,  0.0379, -1.6303,
         2.2994,  0.2167,  0.1570,  0.9954, -0.0946,  0.2199, -0.1479, -0.1624,
         0.0099,  0.1878,  0.5409, -0.0033,  0.2546,  0.2114, -0.2059, -0.0283,
        -1.0402,  1.1989, -0.3712, -0.4924, -0.0010,  0.1025,  0.7438, -0.0608,
         0.1456, -0.0174,  0.2019,  0.1307, -0.1050, -0.0040, -0.5593, -0.5899,
         0.0677,  1.1692, -0.0277,  0.1711,  1.1188,  0.0778, -0.0273,  0.7792,
         0.3692, -0.2778,  0.0304, -0.0945, -0.0294,  0.0026, -0.0552,  0.5237,
        -0.5805,  0.0073, -0.1872,  0.1878,  0.0828, -0.0036,  0.0046, -0.3077,
         0.0552,  0.9907,  0.0605,  0.0657,  0.1772, -0.2496, -0.0221,  0.0194,
        -0.0355, -0.1713,  0.5079, -0.6365,  0.6487,  0.2269,  0.0496, -0.1968,
         0.1782,  0.8922, -0.0258,  0.0619, -0.0213,  0.0691,  0.0069,  0.0202,
         0.4552,  0.1673, -0.0076,  0.20